In [109]:
import json
import random
from collections import Counter

def main():
    #最初に生成するReferring Expressionの不確定性の上限
    MAX_UNCERTAINTY = 5
    open_file_path = '../../data/edited_objects.json'
    out_file_path = '../../data/question_data.json'
    file_open = open(open_file_path,'r')
    file = json.load(file_open)

    tar_n = 990 #リストの何番目の要素をtargetとしているか

    #大枠はリストだから最初に要素数を指定。全部やるときはfor文にする
    target_id = file[tar_n]["id"]
    scene_id = file[tar_n]["scene_id"]
    #print(target_id)

    #それぞれのattributeは階層が１つ深い
    attributes = file[tar_n]["attributes"]

    #targetはランダムで選ばれたattributesだけをref_expリストに格納していく
    #その他の同一シーン内のインスタンスは全attributesをリストに格納する
    #targetのattributesリストを包含するインスタンスのidをdistractorリストに格納していく
    #この一連の流れはできればモジュール化したい（データセットのversionをあげても使いたい）
    target_label = file[tar_n]['label']
    ref_exp = [target_label]
    distractor_list = []

    #要素番号もリストで持つようにする
    distractor_index_list = []

    #ランダムで含ませるAttributesに関するFRAG
    #lexicalとstate,other,style,symmetoryはデータセットv1では無視する
    color_flag = random.randrange(2)
    shape_flag = random.randrange(2)
    size_flag = random.randrange(2)
    material_flag = random.randrange(2)
    texture_flag = random.randrange(2)

    unknown_attributes = []

    if color_flag == 1:
        #colorなしの場合は空リスト
        color = attributes.get('color',[])
    else: color = []
    if shape_flag == 1:
        shape = attributes.get('shape',[])
    else: shape = []
    if size_flag == 1:
        size = attributes.get('size',[])
    else: size = []
    if material_flag == 1:
        material = attributes.get('material',[])
    else: material = []
    if texture_flag == 1:
        texture = attributes.get('texture',[])
    else: texture = []

    #Referring Expressionに含まれない
    if color == []:
        unknown_attributes.append('color')
    if shape == []:
        unknown_attributes.append('shape')
    if size == []:
        unknown_attributes.append('size')
    if material == []:
        unknown_attributes.append('material')
    if texture == []:
        unknown_attributes.append('texture')

    ref_exp.extend(color)
    ref_exp.extend(shape)
    ref_exp.extend(size)
    ref_exp.extend(material)
    ref_exp.extend(texture)

    print("シーンID:", scene_id)
    print("ターゲットID:", target_id)
    print("Referring Expression:", ref_exp)
    ref_detail = len(ref_exp)

    #のちに包含か調べるためにset型に変換
    ref_exp_set = set(ref_exp)
    start = False
    for object in file:
        if object["scene_id"] == scene_id:
            start = True
        #scene_idが同じやつは必ず連続なのでforの途中で違くなったらbreakして大丈夫
        if start: 
            if object["scene_id"] != scene_id:
                break
                
            #ここの部分無駄かも
            object_id = object["id"]
            object_label = object["label"]
            Obj_Exp = [object_label]
            obj_attributes = object["attributes"]
            obj_color = obj_attributes.get('color',[])
            obj_shape = obj_attributes.get('shape',[])
            obj_size = obj_attributes.get('size',[])
            obj_material = obj_attributes.get('material',[])
            obj_texture = obj_attributes.get('texture',[])
        
            Obj_Exp.extend(obj_color)
            Obj_Exp.extend(obj_shape)
            Obj_Exp.extend(obj_size)
            Obj_Exp.extend(obj_material)
            Obj_Exp.extend(obj_texture)

            Obj_Exp_Set = set(Obj_Exp)

            #print(ref_exp_Set <= Obj_Exp_Set)
            if ref_exp_set <= Obj_Exp_Set:
                #print(object_id)
                distractor_list.append(object_id)
                distractor_index_list.append(file.index(object))
            
    print("条件を満たすインスタンスのIDリスト:",distractor_list)
    print("条件を満たすインスタンスの要素番号リスト:",distractor_index_list)
    uncertainty = len(distractor_list) - 1
    print("不確定性:", uncertainty)

    #distractorリストの大きさによって初期REを削除
    #最後に同一シーンでREの重複があれば削除する
    if uncertainty <= MAX_UNCERTAINTY:
        ##Noneのあつかい決める（何かしらの重み付をしたほうが良さそう）
        #uncertainty==0の場合はNo qestions
        if uncertainty == 0:
            q_attribute = 'None'
            print('質問内容：',q_attribute)
        else:

            #どのattributeを対象として分割すると不確定性の期待値が最小となるかを計算

            #textureを質問しやすくなるのを防ぐために要素をシャッフル
            random.shuffle(unknown_attributes)
            print("不確定要素",unknown_attributes)
            min = uncertainty
            for unknown_attribute in unknown_attributes:
                #最終的にminと不確定性が等しかったらどんな質問をした場合も不確定性を減らせないということになる
                
                #注目しているattributeを条件を満たすインスタンスリストから全部引っ張ってくる.
                #一つのattributeに要素を複数含む場合にどうするか
                #現在は不確定性の期待値を計算する際に分母を全要素数にすることで対応
                #持たない場合はNone
                tmp = []
                for index in distractor_index_list:
                    tmp.extend(file[index]['attributes'].get(unknown_attribute,['None']))
                
                tmp_c = Counter(tmp)
                print(unknown_attribute,':',tmp_c.most_common())
                #内包表現
                tmp_sum = [row[1] for row in tmp_c.most_common()]
                #print(tmp_sum)
                #map()は第1引数で定義したラムダ式を第2引数のリストの全要素に適用させるlist()はそれをリストとして返す
                future_uncertainty = sum(list(map(lambda x:x**2, tmp_sum)))/(len(tmp)) - 1
                print('質問した場合の不確定性の期待値：',future_uncertainty)
                if future_uncertainty <= min:
                    min = future_uncertainty
                    q_attribute = unknown_attribute
            if min==uncertainty:    
                print('不確定性を減らせる質問なし')
            else:
                print('質問内容：',q_attribute)
                #min==uncertaintyの場合はブレイク
                #q_attributeをREに追加する
                
                ref_exp.extend(file[tar_n]['attributes'].get(q_attribute,[]))
                next_ref_exp = ref_exp
        
                if len(next_ref_exp) == ref_detail:
                    print("質問の回答がNoneのため次の質問を生成しない")
                else:
                    print("質問後のReferring Expression:",ref_exp)

    else:
        print('不確定性が大きすぎるためReferring Expressionを削除')
        #とりあえず質問の重複は気にせず進める
        #表現方法も自然言語では無くLISTのまま進める
        #全オブジェクトに対して最初のReferring Expressionを生成
        #dict型に変換
        #ref_expとfileとdictを引数で受け取って不確定性が減らなくなるまで再帰してdictを書き換えて返すメソッドの定義

main()

シーンID: ab835fae-54c6-29a1-995e-b06cfc555786
ターゲットID: 18
Referring Expression: ['door', 'rectangular', 'flat']
条件を満たすインスタンスのIDリスト: ['5', '10', '18']
条件を満たすインスタンスの要素番号リスト: [977, 982, 990]
不確定性: 2
不確定要素 ['size', 'color', 'material', 'texture']
size : [('None', 2), ('narrow', 1)]
質問した場合の不確定性の期待値： 0.6666666666666667
color : [('white', 2), ('black', 1)]
質問した場合の不確定性の期待値： 0.6666666666666667
material : [('wooden', 3)]
質問した場合の不確定性の期待値： 2.0
texture : [('None', 3)]
質問した場合の不確定性の期待値： 2.0
質問内容： color
質問後のReferring Expression: ['door', 'rectangular', 'flat', 'black']
